In [ ]:
from IPython.core.display import HTML
from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
plt.style.use('fivethirtyeight')

import pandas as pd
import zipfile
import io
import math

import folium
import json
import pandas as pd
import folium.colormap as cm

def css_styling():
    styles = open('../notebook_styles.css', 'r').read()
    return HTML(styles)
css_styling()

In [ ]:
#Loading testing data
from client.api.notebook import Notebook 
lab07 = Notebook('lab07.ok')
_ = lab07.auth(inline=True)

# Lab 07 - Migration and space

## Introductions

**What is your partner's name?**

[ANSWER HERE]

**What is your partner's favorite place on campus?**

[ANSWER HERE]

**What was your partner's favorite class in high school?**

[ANSWER HERE]

## Migration in the US

So far, we have discussed mortality and fertility.  The third and last major component of demographic change is migration, our subject today.

Although migration is very important for understanding populations and how they change, it is probably the element of demographic change that is least understood. Why?  Data on migration are very hard to get. Migration inherently involves people changing location, and different locations invariably have different authorities who gather and produce population statistics. It can be quite difficult to reconstruct even reasonably basic features of migration.

Nonetheless, we'll be working with data today! We'll explore some aspects of migration within the United States, by looking at county to county flows of people over the course of a year. Our data will come from the [Internal Revenue Service](https://www.irs.gov/uac/soi-tax-stats-migration-data-2014-2015). We'll be looking at migration from 2014-2015.

The [documentation for the IRS migration data](https://www.irs.gov/pub/irs-soi/1415inpublicmigdoc.pdf) has some more information about how these data were produced.

**Question - The IRS dataset has information on, among other things, county to county migration flows. Look at the documentation for the IRS data. How exactly are these flows defined?**

[ANSWER HERE]

**Question - Given the definition of county to county flows, what are the limitations of studying internal US migration using the IRS data? (I.e., who might not show up in the IRS data? What other problems might the data have?)**

[ANSWER HERE]

Some [additional documentation](https://www.irs.gov/statistics/soi-tax-stats-migration-data) tells us that the number of returns filed can be considered an approximation of the number of households who move, and the number of exemptions can be considered an approximation of the number of people who move.

The number of returns is called `n1` in the tables below, and the number of exemptions is called `n2`. We'll focus on `n2`, the number of exemptions, in lab today.

In [ ]:
irs_mig = Table.read_table('../data/IRS/us_migration_14to15_irs.csv')
irs_mig

The first thing to notice about these data is that the counties have an ID called a FIPS code. You can see the FIPS code for all of the counties in the US by looking at a
[FIPS lookup table](https://www.nrcs.usda.gov/wps/portal/nrcs/detail/national/home/?cid=nrcs143_013697).
Keep a tab with this FIPS lookup table open as you work on this lab; you may find it helpful to refer to it below.

We'll also be interested in the distance between different counties, so we'll join some data about these distances to the migration info. Here's a datset with some info on county to county distances (this is a subset of the [county distance database](http://www.nber.org/data/county-distance-database.html) which is provided on the NBER website):

In [ ]:
irs_dist = Table.read_table('../data/IRS/irs_mig_county_distances.csv')
irs_dist

Because we are joining on two columns, we need to use Pandas to do the join. We haven't talked about this in class, or in Data 8, so don't worrry if the code below isn't completely familiar.

In [ ]:
irs_mig = Table().from_df(irs_mig.to_df().join(irs_dist.to_df(), how='left', rsuffix='2')).drop(['from_fips2', 'to_fips2'])
irs_mig

In order to get a sense for how the amount of migration flow is related to the size of each county, we'll also want information about the size of each county. So we'll open up this dataset, which has [US Census population estimates by county](https://www.census.gov/data/tables/2017/demo/popest/counties-total.html):

In [ ]:
county_pops = Table.read_table('../data/us-census/us-county-pops-2015.csv')
county_pops

It will be helpful to have a dataset that has the fips code and the name of each county in the US.  This code chunk makes a `fips_names` Table:

In [ ]:
def first_from_col(col):    
    return(col[0])

fips_names = irs_mig.select(['to_fips', 'y2_countyname', 'y2_state']).group('to_fips', first_from_col)
fips_names = fips_names.relabeled(make_array('to_fips', 'y2_countyname first_from_col', 'y2_state first_from_col'), 
                                  make_array('fips', 'county_name', 'state'))
fips_names

Finally, we'll join in the name of the origin state, for completeness (this is missing from the IRS file).

In [ ]:
irs_mig = irs_mig.join('from_fips', fips_names.select('fips', 'state').relabel('state', 'y1_state'), 'fips')
irs_mig

Now we can begin the lab in earnest.

## Looking at county populations in aggregate

Before we investigate migration, let's take a brief look at the spatial distribution of the US population. We'll investigate the population by US county, which is in the `county_pops` dataset:

In [ ]:
county_pops

It looks like these counties vary a lot in size. Let's dig deeper by plotting the distribution of county sizes:

**Question - Make a histogram of the distribution of county sizes.**

In [ ]:
...

Your histogram should confirm that counties vary a great deal in population.  Is there any rhyme or reason to this distribution of county population sizes?

Wachter (in *Essential Demographic Methods*) has suggested that the spatial distribution of populations across administrative areas often tend to fall on a straight line in a *rank-size plot*. We will not study mechanisms that might lead to this pattern in depth, but if you are curious you can check out [Zipf's Law](https://en.wikipedia.org/wiki/Zipf%27s_law) as a starting point. For our purposes, we'd like to see if these county sizes - which seem to vary tremendously - seem to have any kind of structure to them.

For US counties, we can produce a rank-size plot by

* calculating the rank-order of each population's size, with the biggest population ranked 1
* plotting on the x-axis minus the log-rank of each county
* plotting on the y-axis the log of each county's size

The claim is that the resulting plot should be a straight line.

The following function will calculate the rank of the elements of the array that are passed into it:

In [ ]:
# see https://stackoverflow.com/questions/5284646/rank-items-in-an-array-using-python-numpy
def rank_array(array):
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(start=1, stop=len(array)+1)[::-1]
    return(ranks)

rank_array(make_array(10, 3, 65, 1))

**Question - Use the `rank_array` function create a new table, `county_pop_rank`, which is the table `county_pops` with a new column, `pop_rank`, that has the rank of each county's size.**

In [ ]:
county_pop_rank = ...
county_pop_rank

In [ ]:
_ = lab07.grade('test_county_pop_rank')

**Question - Now add two columns to `county_pop_rank`:**
- **one column, `minus_log_rank` with minus the log rank of population size for each county**
- **one column, `log_pop_2015` with the log of the population size for each county**

In [ ]:
county_pop_rank = county_pop_rank.with_column(...,
                                              ...)

In [ ]:
_ = lab07.grade('test_log_county_pop_rank')

**Question - Now make a scatterplot with minus log rank on the x axis and log population size on the y axis**

In [ ]:
...

**Question - Does the relationship seem to be a straight line?**

[ANSWER HERE]

**Question - Des some subset of counties appear to produce a straight line on the plot? (If so, make a scatterplot restricted to that subset of counties as an illustration)**

In [ ]:
...

The next chunk of code defines a function that will help us visualize the quantities on a map.

In [ ]:
us_counties = '../data/us-counties.json'
geo_json_county_data = json.load(open(us_counties))

def map_counties(table, qty, colormap=None):
    """
    table - the Table. must have a 'fips' column with county-level FIPS codes
    qty - the column in the table to map
    """
    
    df = table.to_df()
    df['fips'] = df['fips'].astype(str)
    
    qty_dict = df.set_index('fips')[qty]
    
    if colormap is None:
        cmin = np.percentile(qty_dict, 10)
        cmax = np.percentile(qty_dict, 90)
        colormap = cm.LinearColormap(['Red', 'Orange', 'Yellow', 'Green'], 
                                   vmin=cmin,
                                   vmax=cmax)\

                    .to_step(n=30)
                #.to_step(n=20,
                #         data=nmr_dict,
                #         method='quantiles')

            
    def get_map_color(id):
        if id not in qty_dict.keys():
            return('#00000000')
        else:
            return colormap(qty_dict[id])

    m = folium.Map(location=[39, -96], 
                   #tiles="Stamen Toner",
                   tiles='cartodbpositron',
                   zoom_start=4)

    # add the legend
    colormap.caption = qty
    m.add_child(colormap)
    
    folium.GeoJson(geo_json_county_data,
                   style_function=lambda feature: {
                       'fillColor' : get_map_color(feature['id']),
                       'color' : 'black',
                       'weight' : .5,
                       'opacity' : .8,
                       #'line_opacity' : .3,
                   }).add_to(m)
    

    return(m)

The function `map_counties` can be used by passing it two arguments: the first argument should be a dataset that has a column called `fips`, corresponding to county fips codes; the second argument is the name of the column that has a numeric variable we'd like to visualize across counties on a map.

**Question - use the `map_counties` function to plot county population (`pop_2015`) in the `county_pop_rank` Table.**

In [ ]:
...

**Question - Looking at the map you just created, what do you notice about the physical size of counties in different parts of the country?**

[ANSWER HERE]

**Question Looking again at the map you just created, what do you notice about the spatial distribution of the US population?**

[ANSWER HERE]

# Migration

Now we'll start to investigate migration -- that is, how population changes across space and time.

## An example: out-migrants from Alameda County

One way to look at migration data is to take a particular county and examine the other counties that people move away to.  To help do this, we'd like to write a function, `county_destination_dist`, that shows how many people move from a given a county to each other county (i.e., the distribution of destinations from the given county).

As a part of writing this function, we'd like to rename the columns to make them fit our analysis better. (This is something you'll often want to do as part of data analaysis). Specificaly, we'd like to end up with columns named

* origin_fips
* origin_state
* origin_county
* dest_fips
* dest_state
* dest_county
* num_out_migrants
* mi_to_county

**Question - Fill in the code to complete the `county_destination_dist` function below.**   
*[HINT: Check out the `table_relabel` function for info on renaming columns]*   
*[HINT: remember that the `n2` column has the approximate number of people who move]*

In [ ]:
def county_destination_dist(fips):
    orig = irs_mig.where(..., ...)
    orig = orig.relabel(...,
                        ...)
    
    # select only the relabeled columns, plus also 'mi_to_country' to include in the result
    orig = orig.select(...)
    
    # sort the result in descending order of the number of out migrants to each county
    orig = orig.sort(..., ...)
    return(orig)

cd = county_destination_dist(1001)
cd

In [ ]:
_ = lab07.grade('test_county_dest')

**Question - Use the `county_destination_dist` function to get the destinations for people who migrate away from Alameda County, California**  
*[HINT: You can use the FIPS lookup table linked to above to find the FIPS code for Alameda County]*

In [ ]:
alameda_dest = ...
alameda_dest.show()

**Question - Make a bar plot that shows a bar for each county people move to from Alameda County; the height of each bar should be proportional to the number of people who move to each county.**

In [ ]:
...

Now that we have a function that will give us the distribution of destinations for out-migrants from any county, we can make use of the next function to help us visualize the results:

In [ ]:
def map_out_migrants(fips):
    tab = county_destination_dist(fips)
    sending_fips = tab.column('origin_fips').item(0)
    tot_sent = -1*np.sum(tab['num_out_migrants'])
    tomap = tab.relabel(make_array('dest_fips', 'num_out_migrants'), make_array('fips', 'num')).select('fips', 'num')
    tomap = tomap.with_row([sending_fips, tot_sent])
    
    cmin = np.min(tomap['num'])
    cmax = np.max(tomap['num'])
    colormap = cm.LinearColormap(['Red', 'Green', 'DarkGreen'], 
                                 index=[cmin,0,cmax],
                                 vmin=cmin, vmax=cmax)

    return(map_counties(tomap, 'num', colormap=colormap))

The `map_out_migrants` function takes the fips code for a county, and it produces a map that shows the destinations of outmigrants from that county.

**Question - Use `map_out_migrants` to visualize (make a map) of where out-migrants from Alameda County, CA end up**

In [ ]:
...

**Question - looking at the map you just made, come up with two factors that you think might affect where people from Alameda county decide to migrate**

[ANSWER HERE]

**Question - Investigate the relationship between the number of out migrants from Alameda to each county and the size of each county. Does there appear to be a relationship? Do counties with bigger populations attract more out-migrants?**

In [ ]:
...

**Question - Investigate the relationship between the number of out migrants from Alameda to each county and the distance to each county. Does there appear to be a relationship? Do counties that are closer attract more out-migrants?**   
*[HINT: the column `mi_to_county` has the approximate distance from one county to another in miles]*

In [ ]:
...

**Question - How would you synthesize the previous two results? Do distance and population explain why people migrate to a given county?**

[ANSWER HERE]

## An example, continued: in-migrants to Alameda County

Continuing with the specific example of Alameda County, we'll now turn our previous analysis on its head: instead of investigating who moves away from Alameda County, we'll look at who moves into Alameda County.

To start with, we will want to make a `county_arrivals_dist` function that returns the counties that send people to Alameda County. (This is like the reverse of the `county_destination_dist` function that you wrote above.

Once again, as a part of writing this function, we'd like to rename the columns to make them fit our analysis better. (This is something you'll often want to do as part of data analaysis). Specificaly, we'd like to end up with columns named

* origin_fips
* origin_state
* origin_county
* dest_fips
* dest_state
* dest_county
* num_in_migrants
* mi_to_county

In [ ]:
def county_arrivals_dist(fips):
    ## see the `county_destination_dist` function above for inspiration here
    dest = irs_mig.where(..., ...)
    dest = dest.relabel(..., ...)
    dest = dest.select(...)
    dest = dest.sort(..., ...)    
    return(dest)

ca = county_arrivals_dist(1001)
ca

In [ ]:
_ = lab07.grade('test_county_arrivals')

**Question - Use the `county_arrivals_dist` function to get the destinations for people who migrate into Alameda County, California**  

In [ ]:
alameda_arrivals = ...
alameda_arrivals.show()

In [ ]:
_ = lab07.grade('test_alameda_arrivals')

Like before, we've made a `map_in_migrants` function to help you visualize where migrants are arriving from.

In [ ]:
def map_in_migrants(fips):
    tab = county_arrivals_dist(fips)
    receiving_fips = tab.column('dest_fips').item(0)
    tot_arrived = np.sum(tab['num_in_migrants'])
    tomap = tab.relabel(make_array('origin_fips', 'num_in_migrants'), 
                        make_array('fips', 'num')).select('fips', 'num')
    tomap['num'] = -1*tomap.column('num')
    tomap = tomap.with_row([receiving_fips, tot_arrived])
    
    cmin = np.min(tomap['num'])
    cmax = np.max(tomap['num'])
    colormap = cm.LinearColormap(['DarkRed', 'red', 'DarkGreen'], 
                                 index=[cmin,0,cmax],
                                 vmin=cmin, vmax=cmax)

    return(map_counties(tomap, 'num', colormap=colormap))
    #return(tomap)

`map_in_migrants` takes the fips code for a county and visualizes where migrants to that county arrive from.

**Quesiton - use `map_in_migrants` to visualize where people move to Alameda County from**

In [ ]:
...

## Looking at migration in aggregate

Having developed some tools for investigating in- and out-migration using the specific example of Alameda County, we'll now try to study the phenomenon of in- and out-migration across counties.  Our approach will be to develop rate-like quantities that measure the intensity of in- and out-migration. We'll start with out-migration, which can be measured using an *out-migration rate*:

$$
\text{Out-Migration Rate (OMR)} = \frac{\text{# people moving out of county}}{\text{# people in county}}
$$

The out-migration rate captures how much out-migration is happening, relative to the number of people who could possibly out-migrate.

**Question - Make a table of out-migrants for each county by summing the number of people who move out of each county. (The sum is taken across all of the counties people move to.)**   
*[HINT: Use the `group` function here]*

In [ ]:
out_migrants = irs_mig....

# the resulting Table should have two columns: fips and num_out_migrants
out_migrants = out_migrants.relabel(..., ...)
out_migrants

In [ ]:
_ = lab07.grade('test_out_migrants')

**Question - Add readable county and state names to the `out_migrants` dataset by joining it onto the `fips_names` table.**

In [ ]:
out_migrants = ...
out_migrants

**Question - Finally, add information about county sizes to the `out_migrants` table by joining it onto the `county_pops` table.**

In [ ]:
out_migrants = ...
out_migrants

In [ ]:
_ = lab07.grade('test_out_migrants2')

**Question - Plot the relationship between county population (x axis) and number of out migrants (y axis). Do these two quantities appear to be linked?**

In [ ]:
...

**Question - Now calculate the out-migration rate for each county; add it to the `out_migrants` dataset in a column called `omr`.**

In [ ]:
out_migrants = ...
out_migrants

In [ ]:
_ = lab07.grade('test_omr')

**Question - Make a histogram that has the distribution of out-migration rates across counties.**

In [ ]:
...

You should see that out-migration rates are often greater than 0, with the modal values being between 0.01 and 0.02 (or between 1 and 2 percent).

### In-migration

We can study in-migration in aggregate using a quantity that we'll call the in-migration rate. The in-migratino rate is (sort of) analogous to the out-migration rate:

$$
\text{In-Migration Rate (IMR)} = \frac{\text{# people moving into county}}{\text{# people in county}}
$$

**Question - Most demographers would argue that the in-migration rate does not make as much sense as the out-migraiton rate. What problem do you think most demographers would point out in our definition of the in-migration rate?**   
*[NB: This is slightly tricky - do your best!]*

[ANSWER HERE]

**Question - Following the pattern from our study of out-migration above, make a table with total in-migration to each county.**

In [ ]:
in_migrants = irs_mig....

# the resulting Table should have two columns: fips and num_out_migrants
in_migrants = ...
in_migrants

In [ ]:
_ = lab07.grade('test_in_migrants')

**Question - Add county and state names to your `in_migrants` table by joining `fips_names` onto it.**

In [ ]:
in_migrants = ...
in_migrants

**Question - Add county sizes to your `in_migrants` table by joining `county_pops` onto it.**

In [ ]:
in_migrants = ...
in_migrants

In [ ]:
_ = lab07.grade('test_in_migrants2')

**Question - Plot in-migrant numbers (y axis) against population counts (x axis). Do these two quantities seem related?**

In [ ]:
...

**Question - calculate the in-migration rate and add it to a column of `in_migrants`; call the new column `imr`.**

In [ ]:
in_migrants = in_migrants....
in_migrants

In [ ]:
_ = lab07.grade('test_imr')

**Question - Make a histogram that has the distribution of in-migration rates across counties.**

In [ ]:
...

**Question - Did you find that there was more variation in the out-migration rates or the in-migratino rates?**

[ANSWER HERE]

**More resources for the curious**

The US Census has a [Flows mapper](https://flowsmapper.geo.census.gov/map.html) website, where you can visualize migration from county to county in the US.  
Note that the US Census estimates are from a different source--the American Community Survey--and from a wider range of time-periods than the IRS data. So the two sources may not be exactly the same. The ACS data are probably better, actually, than the IRS data, but they are considerably more complex to work with.

## Challenge question (optional)

If you have extra time, use the visualization tools we developed to explore in- and out-migration patterns for different counties. Can you find any general patterns that are suggestive of a particular social or economic process that causes migration?

## Run all tests

This cell just re-runs all of the unit tests in the notebook, to summarize the results

In [ ]:
# this cell runs all the tests at once!
print("Running all tests...")
_ = [lab07.grade(q[:-3]) for q in os.listdir("tests") if q.startswith('test')]
print("Finished running all tests.")

### Submit your assignment by MIDNIGHT on the day of class

Please submit your lab in by running the cell below. You can submit as many times as you want, up to midnight on the day of the class. No late submissions are allowed, and the system will prevent you from being able to submit late.

In [ ]:
_ = lab07.submit()